In [8]:
import tensorflow as tf

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [26]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')

training = tf.placeholder_with_default(False, shape=(), name='training')

In [27]:
y = tf.placeholder(tf.int64, shape=(None), name='y')

In [4]:
hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1')
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name='hidden2')
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name='outputs')
logits = tf.layers.batch_normalization(logits_before_bn, training=training, momentum=0.9)

In [5]:
import numpy as np 

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [24]:
reset_graph()

In [28]:
from functools import partial

my_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1')
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name='hidden2')
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name='outputs')
logits = my_batch_norm_layer(logits_before_bn)

In [29]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

In [30]:
from tensorflow.examples.tutorials.mnist import input_data
mnist= input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [31]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [32]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [33]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [34]:

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

n_epochs = 40
batch_size = 20

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op, extra_update_ops], feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        
        print(epoch, "Test accuracy:", accuracy_val)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Test accuracy: 0.9411
1 Test accuracy: 0.9575
2 Test accuracy: 0.9627
3 Test accuracy: 0.9697
4 Test accuracy: 0.9727
5 Test accuracy: 0.9722
6 Test accuracy: 0.9768
7 Test accuracy: 0.9758
8 Test accuracy: 0.9777
9 Test accuracy: 0.9772
10 Test accuracy: 0.9786
11 Test accuracy: 0.9792
12 Test accuracy: 0.9794
13 Test accuracy: 0.978
14 Test accuracy: 0.9797
15 Test accuracy: 0.9793
16 Test accuracy: 0.9802
17 Test accuracy: 0.9812
18 Test accuracy: 0.9799
19 Test accuracy: 0.9814
20 Test accuracy: 0.9803
21 Test accuracy: 0.9823
22 Test accuracy: 0.9813
23 Test accuracy: 0.9819
24 Test accuracy: 0.9807
25 Test accuracy: 0.9829
26 Test accuracy: 0.9835
27 Test accuracy: 0.9821
28 Test accuracy: 0.9825
29 Test accuracy: 0.9823
30 Test accuracy: 0.9824
31 Test accuracy: 0.9822
32 Test accuracy: 0.9828
33 Test accuracy: 0.9825
34 Test accuracy: 0.9826
35 Test accuracy: 0.9817
36 Test accuracy: 0.9825
37 Test accuracy: 0.9825
38 Test accuracy: 0.9833
39 Test accuracy: 0.9818


In [23]:
reset_graph()

# transfer_learning
saver = tf.train.import_meta_graph('./my_model_final.ckpt.meta')

for op in tf.get_default_graph().get_operations():
    print(op.name)

X
training/input
training
y
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
hidden1/MatMul
hidden1/BiasAdd
batch_normalization/beta/Initializer/zeros
batch_normalization/beta
batch_normalization/beta/Assign
batch_normalization/beta/read
batch_normalization/gamma/Initializer/ones
batch_normalization/gamma
batch_normalization/gamma/Assign
batch_normalization/gamma/read
batch_normalization/moving_mean/Initializer/zeros
batch_normalization/moving_mean
batch_normalization/moving_mean/Assign
batch_normalization/moving_mean/read
batch_normalization/moving_variance/Initializer/one

In [35]:
reset_graph()

# transfer_learning
saver = tf.train.import_meta_graph('./my_model_final.ckpt.meta')
training = tf.placeholder_with_default(False, shape=(), name='training')

from functools import partial

my_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)

n_hidden2 = 150
n_hidden3 = 20
n_outputs = 10

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden1 = tf.get_default_graph().get_tensor_by_name("Elu:0")

new_hidden2 = tf.layers.dense(hidden1, n_hidden2, name='new_hidden2')
new_bn2 = my_batch_norm_layer(new_hidden2)
new_bn2_act = tf.nn.elu(new_bn2)

new_hidden3 = tf.layers.dense(new_bn2_act, n_hidden3, name='new_hidden3')
new_bn3 = my_batch_norm_layer(new_hidden3)
new_bn3_act = tf.nn.elu(new_bn3)

new_logits_before_bn = tf.layers.dense(new_bn3_act, n_outputs, name='outputs')
new_logits = my_batch_norm_layer(new_logits_before_bn)

with tf.name_scope("new_loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=new_logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("new_eval"):
    correct = tf.nn.in_top_k(new_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("new_train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
new_saver = tf.train.Saver()

In [36]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

n_epochs = 30
batch_size = 20

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op, extra_update_ops], feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        
        print(epoch, "Test accuracy:", accuracy_val)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Test accuracy: 0.9417
1 Test accuracy: 0.9546
2 Test accuracy: 0.9615
3 Test accuracy: 0.9679
4 Test accuracy: 0.9718
5 Test accuracy: 0.9733
6 Test accuracy: 0.9739
7 Test accuracy: 0.9739
8 Test accuracy: 0.9764
9 Test accuracy: 0.977
10 Test accuracy: 0.974
11 Test accuracy: 0.9796
12 Test accuracy: 0.9798
13 Test accuracy: 0.978
14 Test accuracy: 0.981
15 Test accuracy: 0.9797
16 Test accuracy: 0.9793
17 Test accuracy: 0.9819
18 Test accuracy: 0.9809
19 Test accuracy: 0.9819
20 Test accuracy: 0.9802
21 Test accuracy: 0.9809
22 Test accuracy: 0.9808
23 Test accuracy: 0.9815
24 Test accuracy: 0.9801
25 Test accuracy: 0.981
26 Test accuracy: 0.9832
27 Test accuracy: 0.9821
28 Test accuracy: 0.9821
29 Test accuracy: 0.9812


In [19]:
reset_graph()

# transfer_learning
saver = tf.train.import_meta_graph('./my_model_final.ckpt.meta')

for op in tf.get_default_graph().get_operations():
    print(op.name)

X
training/input
training
y
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
hidden1_1/MatMul
hidden1_1/BiasAdd
batch_normalization/beta/Initializer/zeros
batch_normalization/beta
batch_normalization/beta/Assign
batch_normalization/beta/read
batch_normalization/gamma/Initializer/ones
batch_normalization/gamma
batch_normalization/gamma/Assign
batch_normalization/gamma/read
batch_normalization/moving_mean/Initializer/zeros
batch_normalization/moving_mean
batch_normalization/moving_mean/Assign
batch_normalization/moving_mean/read
batch_normalization/moving_variance/Initializer

In [22]:
reset_graph()

# transfer_learning with frozen lower layers
saver = tf.train.import_meta_graph('./my_model_final.ckpt.meta')
training = tf.placeholder_with_default(False, shape=(), name='training')

from functools import partial

my_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)

n_hidden3 = 50
n_hidden4 = 20
n_outputs = 10

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden2 = tf.get_default_graph().get_tensor_by_name("Elu_1:0")

new_hidden3 = tf.layers.dense(hidden2, n_hidden3, name='new_hidden3')
new_bn3 = my_batch_norm_layer(new_hidden3)
new_bn3_act = tf.nn.elu(new_bn3)

new_hidden4 = tf.layers.dense(new_bn3_act, n_hidden4, name='new_hidden4')
new_bn4 = my_batch_norm_layer(new_hidden4)
new_bn4_act = tf.nn.elu(new_bn4)

new_logits_before_bn = tf.layers.dense(new_bn4_act, n_outputs, name='outputs')
new_logits = my_batch_norm_layer(new_logits_before_bn)

with tf.name_scope("new_loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=new_logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("new_eval"):
    correct = tf.nn.in_top_k(new_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("new_train"):
    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|outputs")
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss, var_list=train_vars)

init = tf.global_variables_initializer()
new_saver = tf.train.Saver()

In [23]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

n_epochs = 30
batch_size = 20

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op, extra_update_ops], feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        
        print(epoch, "Test accuracy:", accuracy_val)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Test accuracy: 0.5599
1 Test accuracy: 0.5968
2 Test accuracy: 0.5981
3 Test accuracy: 0.6071
4 Test accuracy: 0.6102
5 Test accuracy: 0.6085
6 Test accuracy: 0.6109
7 Test accuracy: 0.6136
8 Test accuracy: 0.6088
9 Test accuracy: 0.6054
10 Test accuracy: 0.6124
11 Test accuracy: 0.6132
12 Test accuracy: 0.607
13 Test accuracy: 0.6097
14 Test accuracy: 0.6044
15 Test accuracy: 0.6156
16 Test accuracy: 0.5986
17 Test accuracy: 0.6106
18 Test accuracy: 0.6103
19 Test accuracy: 0.6063
20 Test accuracy: 0.6084
21 Test accuracy: 0.6085
22 Test accuracy: 0.6089
23 Test accuracy: 0.6081
24 Test accuracy: 0.6064
25 Test accuracy: 0.6122
26 Test accuracy: 0.6079
27 Test accuracy: 0.6122
28 Test accuracy: 0.6083
29 Test accuracy: 0.6114


In [ ]:
reset_graph()

import numpy as np

n_batches = mnist.train.num_examplesamplesum_examples // batch_size

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, './my_model_final.ckpt')
    
    # cache output of hidden layer 2 at the start
    h2_cache = sess.run(hidden2, feed_dict={X: mnist.train.images})
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(mnist.train.num_examples)
        hidden2_batches = np.array_split(ht_cache[shuffled_idx], n_batches)
        y_batches = np.array_split(mnist.train.labels[shuffled_idx], n_batches)
        for hidden2_batch, y_batch in zip(hidden_batches, y_batches):
            # feed in the cached outputs
            sess.run(training_op, feed_dict={hidden2: hidden2_batch, y: y_batch})
    
    save_path = saver.save(sess, './my_new_mondel_final.ckpt')